# Federated learning: aggregation operators

In this notebook, we provide an explanation of the implementation of the different federated aggregation operators provided in the framework. Before discussing the different aggregation operators, we must establish the federated configuration (for more information see notebook [Federated learning basic concepts](./federated_learning_basic_concepts.ipynb)).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

import shfl
from shfl.private.utils import normalize_query



database = shfl.data_base.Emnist()
train_data, train_labels, test_data, test_labels = database.load_data()

iid_distribution = shfl.data_distribution.IidDataDistribution(database)
nodes_federation, test_data, test_labels = iid_distribution.get_nodes_federation(num_nodes=5, percent=10)

def model_builder():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu', strides=1, input_shape=(28, 28, 1)))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=2, strides=2, padding='valid'))
    model.add(tf.keras.layers.Dropout(0.4))
    model.add(tf.keras.layers.Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu', strides=1))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=2, strides=2, padding='valid'))
    model.add(tf.keras.layers.Dropout(0.3))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(128, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.1))
    model.add(tf.keras.layers.Dense(64, activation='relu'))
    model.add(tf.keras.layers.Dense(10, activation='softmax'))

    loss = tf.keras.losses.CategoricalCrossentropy()
    optimizer = tf.keras.optimizers.RMSprop()
    metrics = [tf.keras.metrics.categorical_accuracy]
    
    return shfl.model.DeepLearningModel(model=model, loss=loss, optimizer=optimizer, metrics=metrics)


def reshape_data(labeled_data):
    labeled_data.data = np.reshape(labeled_data.data, (labeled_data.data.shape[0], labeled_data.data.shape[1], labeled_data.data.shape[2],1))
        
nodes_federation.apply_data_transformation(reshape_data)

mean = np.mean(train_data.data)
std = np.std(train_data.data)
nodes_federation.apply_data_transformation(normalize_query, mean=mean, std=std);

test_data = np.reshape(test_data, (test_data.shape[0], test_data.shape[1], test_data.shape[2],1))

Once we have loaded and federated the data and established the learning model, the only step that remains is to establish the aggregation operator. At the moment, the framework has FedAvg and WeightedFedAvg implemented. The implementation of the federated aggregation operators are as follows.

NOTE: The aggregators are required to be *callable*. That is, we can use a simple function. Alternatively, if a more sophisticated implementation is needed in a class, this has simply to implement the method `__call__` (see code in following links for examples). 

## Federated averaging operator

In this section, we detail the implementation of `FedAvg` (see [FedAvg](https://github.com/sherpaai/Sherpa.ai-Federated-Learning-Framework/blob/master/shfl/federated_aggregator/fedavg_aggregator.py)) proposed  by Google in this [paper](https://arxiv.org/abs/1602.05629). 

It is based on the arithmetic mean of the local weights $W_i$ trained in each of the local clients $C_i$. That is, the weights $W$ of the global model after each round of training are

$$W = \frac{1}{n_{\rm{C}}} \sum_{i=1}^{n_{\rm{C}}} W_i$$


For its implementation, we create a class that implements the [FederatedAggregator](https://github.com/sherpaai/Sherpa.ai-Federated-Learning-Framework/blob/master/shfl/federated_aggregator/federated_aggregator.py) interface.

In [ ]:
from shfl.federated_aggregator.fedavg_aggregator import FedAvgAggregator


fedavg_aggregator = FedAvgAggregator()

## Weighted federated averaging operator

In this section, we detail the implementation of `WeightedFedAvg` (see [WeightedFedAvg](https://github.com/sherpaai/Sherpa.ai-Federated-Learning-Framework/blob/master/shfl/federated_aggregator/weighted_fedavg_aggregator.py)). It is the weighted version of `FedAvg`. The weight of each client $C_i$ is determined by the amount of client data $n_i$ with respect to total training data $n$. That is, the parameters $W$ of the global model after each round of training are:

$$W =  \sum_{i=1}^n \frac{n_i}{n} W_i$$

When all clients have the same amount of data, it is equivalent to FedAvg.

To implement it, we create a class that implements the `FederatedAggregator` interface. For this purpose, we first weigh the local parameters by percentage and then sum the weighted parameters.

In [ ]:
from shfl.federated_aggregator.weighted_fedavg_aggregator import WeightedFedAggregator


weighted_fedavg_aggregator = WeightedFedAggregator()

Finally, we are ready to establish the federated government with any of the implemented aggregation operators and start the federated learning process.

In [ ]:
federated_government = shfl.federated_government.FederatedGovernment(model_builder(), nodes_federation, fedavg_aggregator)

In [ ]:
federated_government.run_rounds(1, test_data, test_labels)

## Cluster federated averaging operator

In this section, we detail the implementation of `ClusterFedAvg` (see [ClusterFedAvg](https://github.com/sherpaai/Sherpa.ai-Federated-Learning-Framework/blob/master/shfl/federated_aggregator/cluster_fedavg_aggregator.py)). 

Cluster Federated Averaging is based on the aggregation operator used for k-means clustering. When aggregating the centroids of a federated K-means clustering, we are faced with the problem of grouping the clusters for subsequent aggregation. Based on the hypothesis that the closest centroids will belong to the same cluster, we apply K-means over the centroids, in order to group the centroids that belong to the same cluster and to obtain the representation (aggregation) of each group. We choose the new centroids obtained as the aggregation.

This time, instead of a class, we implement a function (remember, the aggregator simply needs to be callable):

In [ ]:
from shfl.federated_aggregator.cluster_fedavg_aggregator import cluster_fed_avg_aggregator

clustering_aggregator = cluster_fed_avg_aggregator

We create a federated government of clustering, in order to apply this aggregation operator.

In [ ]:
c_database = shfl.data_base.Iris()
c_train_data, c_train_labels, c_test_data, c_test_labels = c_database.load_data()

c_iid_distribution = shfl.data_distribution.IidDataDistribution(c_database)
c_nodes_federation, c_test_data, c_test_labels = c_iid_distribution.get_nodes_federation(num_nodes=3, percent=50)

n_clusters = 3 # Set number of clusters
n_features = train_data.shape[1]
def clustering_model_builder():
    model = shfl.model.KMeansModel(n_clusters=n_clusters, n_features = n_features)
    return model

clustering_federated_government = shfl.federated_government.FederatedGovernment(clustering_model_builder(), c_nodes_federation, clustering_aggregator)

In [ ]:
clustering_federated_government.run_rounds(1, c_test_data, c_test_labels)